In [1]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import ast
import numpy as np
import os
import ast
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os.path
from datetime import datetime
from collections import Counter

nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/domantas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/domantas/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /home/domantas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/domantas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Single URL prediction

In [2]:
month = "january"
# os.chdir("/home/domantas/Desktop/University/bachelor_url_categorization")

from sklearn.externals import joblib

filename = "../Models/Models_LR_2019-05-05.joblib"
lr = joblib.load(filename) 
# filename = "../Models/{}/LSVM_model_{}.joblib".format(month.title(), month)
# lsvm = joblib.load(filename) 
import pickle
filename = "../Frequency_models/word_frequency_2019-05-05.picle"
pickle_in = open(filename,"rb")
words_frequency = pickle.load(pickle_in)
df = pd.read_csv("../Datasets/Testing_dataset.csv")[['URL', 'Category']]
df = df[df['URL'].notnull()]

FileNotFoundError: File b'../Datasets/Testing_dataset.csv' does not exist

In [7]:
df.head()

Unnamed: 0         Country   Retailer  \
0           0             USA    benefit   
1           1  United Kingdom    benefit   
2           2             USA  MyProtein   
3           3  United Kingdom  MyProtein   
4           4             USA       Zara   

                                             Website            Category  \
0  https://www.benefitcosmetics.com/us/en/shop-ma...  Beauty_and_Fitness   
1  https://www.benefitcosmetics.com/uk/en-gb/shop...  Beauty_and_Fitness   
2                          https://us.myprotein.com/  Beauty_and_Fitness   
3                         https://www.myprotein.com/  Beauty_and_Fitness   
4                           https://www.zara.com/us/            Shopping   

   English_cof                                          Tokens_en  \
0    64.235294  ['shop', 'benefit', 'cosmetic', 'skip', 'main'...   
1    67.629362  ['cosmetic', 'product', 'benefit', 'cosmetic',...   
2    63.591636  ['supplement', 'sport', 'nutrition', 'skip', '...   
3    66.305656  ['sport', 'nutrition', 'clothing', 'brand', 's...   
4    77.821782  ['united', 'state', 'new', 'collection', 'ente...   

                                              Tokens  
0  ['shop', 'makeup', '', 'benefit', 'cosmetic', ...  
1  ['makeup', '', 'cosmetic', 'product', '', 'ben...  
2  ['myprotein™', '', 'bodybuilding', 'supplement...  
3  ['myprotein™', '', 'sport', 'nutrition', '', '...  
4  ['zara', 'united', 'state', '', 'new', 'collec...

### Downloading webpages and creating my model

In [9]:
toker = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps=True)
char_blacklist = list(chr(i) for i in range(32, 127) if i <= 64 or i >= 91 and i <= 96 or i >= 123)
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(char_blacklist)
top = 2500
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}
row_counter = 0 
for row_id, row in df.iterrows():
    row_counter += 1
    try:
        url = row['URL']
        req = urllib.request.Request(url, headers=hdr)
        html = urlopen(req, timeout=15).read()
        # html = urlopen(url, timeout=15).read()
        soup = BeautifulSoup(html, "html.parser")
        [tag.decompose() for tag in soup("script")]
        [tag.decompose() for tag in soup("style")]
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk.lower() for chunk in chunks if chunk)
        tokens = nltk.word_tokenize(text)
        wnl = WordNetLemmatizer()
        tokens_lemmatize = [wnl.lemmatize(token) for token in tokens]
        
        from collections import Counter
        counter = 0
        features_pred = np.zeros(top * len(words_frequency)).reshape(len(words_frequency), top)
        c = [word for word, word_count in Counter(tokens_lemmatize).most_common(top)]
        for category in words_frequency.keys():
            for word in c:
                if word in words_frequency[category]:
                    features_pred[counter][words_frequency[category].index(word)] = 1
            counter+=1

        category_weight = []
        for i in features_pred:
            weight_cof = np.where(i == 1)[0]
            weight_sum = 0
            for cof in weight_cof:
                weight_sum += top - cof
            category_weight.append(weight_sum)

        cat_index = category_weight.index(max(category_weight))
        category = list(words_frequency.keys())[cat_index]
        feature = features_pred[cat_index].reshape(-1, top)
        print("url: {} . {} / {}".format(url, row_counter, len(df)))
        print('Category: ', row['Category'])
        print("My model: ",category)
        lr_prediction = lr.predict(feature)
        print("LR prediction: ", list(words_frequency.keys())[int(lr_prediction[0])])
        lsvm_prediction = lsvm.predict(feature)
        print("LSVM prediction: ", list(words_frequency.keys())[int(lsvm_prediction[0])])
        df.at[row_id, 'Weight_model'] = category
        df.at[row_id, 'lr_pred'] = list(words_frequency.keys())[int(lr_prediction[0])]
        df.at[row_id, 'lsvm_pred'] = list(words_frequency.keys())[int(lsvm_prediction[0])]
    except:
        print("{} - Failed. {} / {}".format(row['URL'], row_counter, len(df)))
        continue

url: http://www.bbc.com/ . 1 / 282
Category:  News_and_Media
My model:  News_and_Media
LR prediction:  News_and_Media
LSVM prediction:  News_and_Media
url: https://www.telegraph.co.uk/news/ . 2 / 282
Category:  News_and_Media
My model:  News_and_Media
LR prediction:  News_and_Media
LSVM prediction:  News_and_Media
url: https://news.sky.com/world . 3 / 282
Category:  News_and_Media
My model:  News_and_Media
LR prediction:  News_and_Media
LSVM prediction:  News_and_Media
url: https://www.thesun.co.uk/news/ . 4 / 282
Category:  News_and_Media
My model:  News_and_Media
LR prediction:  News_and_Media
LSVM prediction:  News_and_Media
url: https://www.dailystar.co.uk/news . 5 / 282
Category:  News_and_Media
My model:  News_and_Media
LR prediction:  News_and_Media
LSVM prediction:  News_and_Media
url: https://www.express.co.uk/news . 6 / 282
Category:  News_and_Media
My model:  News_and_Media
LR prediction:  News_and_Media
LSVM prediction:  News_and_Media
url: https://www.theguardian.com/world

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


url: https://www.dailypuppy.com/ . 120 / 282
Category:  Pets_and_Animals
My model:  Home_and_Garden
LR prediction:  Beauty_and_Fitness
LSVM prediction:  Reference
url: https://www.petmd.com/ . 121 / 282
Category:  Pets_and_Animals
My model:  Pets_and_Animals
LR prediction:  Law_and_Government
LSVM prediction:  Law_and_Government
url: https://www.petco.com/shop/en/petcostore . 122 / 282
Category:  Pets_and_Animals
My model:  Pets_and_Animals
LR prediction:  Pets_and_Animals
LSVM prediction:  Pets_and_Animals
url: https://www.sciencedaily.com/ . 123 / 282
Category:  Science
My model:  Science
LR prediction:  Science
LSVM prediction:  Science
url: http://discovermagazine.com/ . 124 / 282
Category:  Science
My model:  Science
LR prediction:  Science
LSVM prediction:  Science
url: https://gizmodo.com/tag/science . 125 / 282
Category:  Science
My model:  Science
LR prediction:  Health
LSVM prediction:  Health
url: https://www.livescience.com/ . 126 / 282
Category:  Science
My model:  Games
L

In [10]:
df = df[df['Weight_model'] != '']
model_acc = len(df[df['Weight_model'] == df['Category']]) / len(df) * 100
lr_acc = len(df[df['lr_pred'] == df['Category']]) / len(df) * 100
lsvm_acc = len(df[df['lsvm_pred'] == df['Category']]) / len(df) * 100
print("My model accuracy: {}% | {} / {}".format(model_acc, len(df[df['Weight_model'] == df['Category']]), len(df)))
print("LR accuracy: {}% | {} / {}".format(lr_acc, len(df[df['lr_pred'] == df['Category']]), len(df)))
print("LSVM accuracy: {}% | {} / {}".format(lsvm_acc, len(df[df['lsvm_pred'] == df['Category']]), len(df)))

My model accuracy: 70.2127659574468% | 198 / 282
LR accuracy: 57.446808510638306% | 162 / 282
LSVM accuracy: 53.90070921985816% | 152 / 282


In [11]:
for category in list(set(df['Category'].values)):
    print(category)
    df2 = df[df['Category'] == category]
    result_w = len(df2[df2['Weight_model'] == category])
    result_lr = len(df2[df2['lr_pred'] == category])   
    result_lsvm = len(df2[df2['lsvm_pred'] == category])
#     print("{} / {} : {:.2f} \% & {} / {} : {:.2f} \% & {} / {} : {:.2f} \%".format(result_w, len(df2), result_w / len(df2) * 100,
#                                                                                       result_lr, len(df2), result_lr / len(df2) * 100, 
#                                                                                       result_lsvm, len(df2), result_lsvm / len(df2) * 100,
#                                                                                      ))
    print("Model: {} / {} : {:.2f} \%".format(result_w, len(df2), result_w / len(df2) * 100))
    print("lr: {} / {} : {:.2f} \%".format(result_lr, len(df2), result_lr / len(df2) * 100))
    print("lsvm: {} / {} : {:.2f} \%".format(result_lsvm, len(df2), result_lsvm / len(df2) * 100))
    print("*"*50)

Shopping
Model: 6 / 7 : 85.71 \%
lr: 5 / 7 : 71.43 \%
lsvm: 3 / 7 : 42.86 \%
**************************************************
Pets_and_Animals
Model: 9 / 12 : 75.00 \%
lr: 5 / 12 : 41.67 \%
lsvm: 5 / 12 : 41.67 \%
**************************************************
Autos_and_Vehicles
Model: 13 / 16 : 81.25 \%
lr: 13 / 16 : 81.25 \%
lsvm: 11 / 16 : 68.75 \%
**************************************************
Reference
Model: 1 / 10 : 10.00 \%
lr: 1 / 10 : 10.00 \%
lsvm: 1 / 10 : 10.00 \%
**************************************************
Gambling
Model: 9 / 9 : 100.00 \%
lr: 6 / 9 : 66.67 \%
lsvm: 5 / 9 : 55.56 \%
**************************************************
Health
Model: 9 / 11 : 81.82 \%
lr: 6 / 11 : 54.55 \%
lsvm: 6 / 11 : 54.55 \%
**************************************************
Food_and_Drink
Model: 6 / 7 : 85.71 \%
lr: 6 / 7 : 85.71 \%
lsvm: 6 / 7 : 85.71 \%
**************************************************
Books_and_Literature
Model: 7 / 10 : 70.00 \%
lr: 7 / 10 : 70.00 

In [ ]:
df.columns

In [ ]:
df[['URL', 'Category','lr_pred', 'lsvm_pred']].to_csv('../Datasets/websites_custom_latex.csv')

In [ ]:
url = "https://www.autotrader.com/"
char_blacklist = list(chr(i) for i in range(32, 127) if i <= 64 or i >= 91 and i <= 96 or i >= 123)
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(char_blacklist)
toker = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps=True)
top = 2500
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}
req = urllib.request.Request(url, headers=hdr)
html = urlopen(req, timeout=15).read()
# html = urlopen(url, timeout=15).read()
soup = BeautifulSoup(html, "html.parser")
[tag.decompose() for tag in soup("script")]
[tag.decompose() for tag in soup("style")]
text = soup.get_text()
lines = (line.strip() for line in text.splitlines())
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
text = '\n'.join(chunk.lower() for chunk in chunks if chunk)
wnl = WordNetLemmatizer()
tokens = [token.lower() for token in toker.tokenize(text)]
tokens_stopwords = [w.lower() for w in tokens if w not in stopwords and len(w) >= 3 and w[0] not in char_blacklist]
tokens_lemmatize = [wnl.lemmatize(token) for token in tokens_stopwords]
allWordExceptStopDist = nltk.FreqDist(w.lower() for w in tokens_lemmatize)
most_common = allWordExceptStopDist.most_common(top)

# My model creation
from collections import Counter
counter = 0
features_pred = np.zeros(top * len(words_frequency)).reshape(len(words_frequency), top)
c = [word for word, word_count in most_common]
for category in words_frequency.keys():
    for word in c:
        if word in words_frequency[category]:
            features_pred[counter][words_frequency[category].index(word)] = 1
    counter+=1

category_weight = []
for i in features_pred:
    weight_cof = np.where(i == 1)[0]
    weight_sum = 0
    for cof in weight_cof:
        weight_sum += top - cof
    category_weight.append(weight_sum)

cat_index = category_weight.index(max(category_weight))
category = list(words_frequency.keys())[cat_index]
feature = features_pred[cat_index].reshape(-1, top)
print("My model: ",category)
prediction = lr.predict(feature)
print("LR normal: ", list(words_frequency.keys())[int(prediction[0])])

In [ ]:
most_common

In [ ]:
df[df['Weight_model'] != df['Category']][["URL", "Category", "Weight_model"]].values